In [1]:
import os
import glob
import shutil
import zipfile

import tensorflow as tf
import numpy as np

import cv2

In [4]:
with zipfile.ZipFile('/content/my_test_image.zip', 'r') as target_file:
  target_file.extractall('/content/my_test_images')

In [3]:

from tensorflow.keras.models import Model, load_model

In [7]:
try:
  cnn = load_model('/content/cifar10_accuracy_82v1.h5')
  cnn.summary()
except Exception as err:
  print(err)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 32, 32, 32)        896       
                                                                 
 conv2d_13 (Conv2D)          (None, 32, 32, 32)        9248      
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 16, 16, 32)       0         
 2D)                                                             
                                                                 
 dropout_11 (Dropout)        (None, 16, 16, 32)        0         
                                                                 
 conv2d_14 (Conv2D)          (None, 16, 16, 64)        18496     
                                                                 
 conv2d_15 (Conv2D)          (None, 16, 16, 64)        36928     
                                                      

In [8]:
images_list = glob.glob('/content/my_test_images/*/*')

In [9]:
images_list

['/content/my_test_images/automobile/automobile.1.jpg',
 '/content/my_test_images/airplane/airplane.1.jpg',
 '/content/my_test_images/horse/horse.1.jpg',
 '/content/my_test_images/ship/ship.1.jpg',
 '/content/my_test_images/deer/deer.1.jpg',
 '/content/my_test_images/dog/dog.1.png',
 '/content/my_test_images/frog/frog.1.jpg',
 '/content/my_test_images/truck/truck.1.jpg',
 '/content/my_test_images/cat/cat.1.jpg',
 '/content/my_test_images/cat/cat.2.jpg']

In [10]:
src_img_list = []
dst_img_list = []

for index in images_list:
  src_img = cv2.imread(index, cv2.IMREAD_COLOR)
  dst_img = cv2.cvtColor(src_img, cv2.COLOR_BGR2RGB)
  dst_img = cv2.resize(dst_img, dsize = (32, 32))
  dst_img = dst_img.astype(np.float32) / 255.0

  src_img_list.append(src_img)
  dst_img_list.append(dst_img)

In [11]:
x_test = np.array(dst_img_list)

In [12]:
x_test.shape

(10, 32, 32, 3)

In [13]:
pred = cnn.predict(x_test)

print(pred.shape)

(10, 10)


In [14]:
class_name = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [26]:
answer_list = []
for index in range(len(images_list)):
  answer = images_list[index].split('/')[4].split('.')[0]
  print(answer)
  answer_list.append(answer)

automobile
airplane
horse
ship
deer
dog
frog
truck
cat
cat


In [22]:
images_list[0].split('/')[4].split('.')[0]

'automobile'

In [28]:
for i in range(len(pred)):
  index = np.argmax(pred[i])
  print(i, 'predict :', class_name[index], ', answer :', answer_list[index]) 

0 predict : truck , answer : cat
1 predict : airplane , answer : automobile
2 predict : bird , answer : horse
3 predict : ship , answer : cat
4 predict : frog , answer : frog
5 predict : dog , answer : dog
6 predict : frog , answer : frog
7 predict : truck , answer : cat
8 predict : cat , answer : ship
9 predict : cat , answer : ship


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize = (10, 4))

for i in range(len(x_test)):
  index = np.argmax(pred[i])

  plt.subplot(2, 5, i+1)
  plt.imshow(x_test[i])
  plt.xticks([]); plt.yticks([])
  plt.title('predict : ' + str(class_name[index]))
  plt.text(5, 0, 'answer : ' + str(answer_list[i]))
plt.tight_layout()
plt.show()

In [43]:
for index in range(len(pred)):

  class_index = np.argmax(pred[index])
  print('prediction => ', class_name[class_index], pred[index].max())

prediction =>  truck 0.50861996
prediction =>  airplane 0.99951756
prediction =>  bird 0.44065484
prediction =>  ship 0.9960563
prediction =>  frog 0.8590545
prediction =>  dog 0.49567932
prediction =>  frog 0.97000724
prediction =>  truck 0.55893034
prediction =>  cat 0.8946541
prediction =>  cat 0.38814557


In [42]:
top3 = 3
for index in range(len(pred)):

  sorted_index = pred[index].argsort()
  sorted_index = sorted_index[::-1]

  print('=' * 30)
  print(sorted_index)

  for j in range(top3):
    pred_val = pred[index, sorted_index[j]]
    class_index = sorted_index[j]

    print('prediction => ', class_name[class_index], pred_val)

[9 1 0 8 6 3 2 5 7 4]
prediction =>  truck 0.50861996
prediction =>  automobile 0.4913783
prediction =>  airplane 1.2694372e-06
[0 2 8 3 9 4 1 6 5 7]
prediction =>  airplane 0.99951756
prediction =>  bird 0.0002905705
prediction =>  ship 0.00019096266
[2 7 0 3 6 5 4 9 8 1]
prediction =>  bird 0.44065484
prediction =>  horse 0.29750487
prediction =>  airplane 0.07216573
[8 0 9 6 1 3 2 4 7 5]
prediction =>  ship 0.9960563
prediction =>  airplane 0.0029145142
prediction =>  truck 0.0005219638
[6 4 3 2 5 7 9 0 1 8]
prediction =>  frog 0.8590545
prediction =>  deer 0.11379467
prediction =>  cat 0.020658279
[5 3 6 4 2 7 9 1 0 8]
prediction =>  dog 0.49567932
prediction =>  cat 0.23136324
prediction =>  frog 0.098725915
[6 3 2 4 5 0 8 7 9 1]
prediction =>  frog 0.97000724
prediction =>  cat 0.021124817
prediction =>  bird 0.006228713
[9 1 0 8 6 3 2 7 5 4]
prediction =>  truck 0.55893034
prediction =>  automobile 0.4410696
prediction =>  airplane 3.065059e-08
[3 2 6 5 4 7 0 8 9 1]
prediction =